In [1]:
import os
import time
import subprocess

# Task 1: Process Creation Utility
def task1_process_creation(n=3):
    print(f"\n[TASK 1] Creating {n} child processes...\n")
    pids = []

    for i in range(n):
        pid = os.fork()
        if pid == 0:
            # Child process
            print(f"Child {i+1}:")
            print(f" PID = {os.getpid()}")
            print(f" Parent = {os.getppid()}")
            print(f" Message = Hello from child {i+1}!\n")
            os._exit(0) # Use os._exit(0) for child to exit immediately
        else:
            pids.append(pid)

    # Parent waits for all
    for pid in pids:
        finished_pid, status = os.waitpid(pid, 0)
        print(f"Parent: Child {finished_pid} finished (status={status})")
    
    print("\nAll child processes completed.\n")

In [5]:
# Task 2: Command Execution Using exec()
def task2_command_exec(commands=None):
    if commands is None:
        commands = [["ls", "-l"], ["date"], ["ps", "aux"]]

    print("\n[TASK 2] Executing commands from child processes using execvp()\n")

    for i, cmd in enumerate(commands):
        pid = os.fork()
        if pid == 0:
            # Child replaces itself with command execution
            print(f"Child {i+1} executing command: {' '.join(cmd)}")
            try:
                os.execvp(cmd[0], cmd)
            except Exception as e:
                print(f"Error executing {cmd[0]}: {e}")
                os._exit(1)
        else:
            # Parent waits for this child
            os.waitpid(pid, 0)

    print("\nAll commands executed by child processes.\n")

In [6]:
# Task 3: Zombie & Orphan Processes
def task3_zombie_and_orphan():
    print("\n[TASK 3] Demonstrating Zombie and Orphan Processes\n")

    # Zombie demonstration
    pid = os.fork()
    if pid == 0:
        print(f"[Zombie Child] PID={os.getpid()} exiting immediately...")
        os._exit(0)
    else:
        print(f"[Parent] Created zombie child PID={pid}, sleeping 5 seconds...")
        print("During this sleep, zombie will exist (Use: ps -el | grep defunct)")
        time.sleep(5)
        os.waitpid(pid, 0)
        print(f"[Parent] Reaped zombie child {pid}\n")

    # Orphan demonstration
    pid = os.fork()
    if pid == 0:
        print(f"[Orphan Child] PID={os.getpid()} started. Parent={os.getppid()}")
        print("[Orphan Child] Sleeping for 5 seconds (parent will exit)...")
        time.sleep(5)
        # Parent PID changes to 1 (init) or another process manager
        print(f"[Orphan Child] Now adopted by init (new parent={os.getppid()})")
        os._exit(0)
    else:
        print(f"[Parent] Exiting before child {pid} finishes (to orphan it).")
        os._exit(0) # Parent exits early to create orphan

In [7]:
# Task 4: Inspecting Process Info from /proc/[pid]/
def task4_inspect_proc(pid):
    print(f"\n[TASK 4] Inspecting process info for PID={pid}\n")

    # Read /proc/[pid]/status
    try:
        with open(f"/proc/{pid}/status") as f:
            lines = f.readlines()
        
        for line in lines:
            # Check for Name, State, and VmRSS
            if any(keyword in line for keyword in ["Name:", "State:", "VmRSS:"]):
                print(line.strip())
    except Exception as e:
        print(f"Error reading /proc/{pid}/status: {e}")

    # Read /proc/[pid]/exe
    try:
        exe = os.readlink(f"/proc/{pid}/exe")
        print(f"Executable Path: {exe}")
    except Exception as e:
        print(f"Error reading /proc/{pid}/exe: {e}")

    # Read /proc/[pid]/fd
    try:
        fds = os.listdir(f"/proc/{pid}/fd")
        print(f"Open File Descriptors: {fds}")
    except Exception as e:
        print(f"Error reading /proc/{pid}/fd: {e}")

In [13]:
# Helper function for CPU-intensive task
def cpu_intensive_task(limit=10000000):
    s = 0
    for i in range(limit):
        s += i # Simple, non-optimizable calculation
    return s

# Task 5: Process Prioritization using nice()
def task5_prioritization(children_n=3):
    print(f"\n[TASK 5] Creating {children_n} CPU-intensive child processes with different priorities\n")
    
    for i in range(children_n):
        pid = os.fork()
        if pid == 0:
            nice_val = i * 5 # Different nice levels: 0, 5, 10, ...
            try:
                # Set process priority using os.nice()
                os.nice(nice_val)
            except PermissionError:
                print(f"Child {i+1}: Insufficient permission to change nice value.")
            
            start = time.time()
            # Perform CPU-intensive task
            cpu_intensive_task(3000000)
            end = time.time()
            
            print(f"Child {i+1} PID={os.getpid()} nice={nice_val} duration={(end-start):.3f} s")
            os._exit(0)
    
    # Parent waits for all children
    for _ in range(children_n):
        os.wait()